This notebook is written for transformers==**4.23.1**, torch==**1.12.1+cu102**, and torchaudio==**0.12.1+cu102**; update your packages if needed.

[ASR Inference with CTC Decoder](https://pytorch.org/audio/main/tutorials/asr_inference_with_ctc_decoder_tutorial.html#acoustic-model-and-set-up)<br>
[Source Code for audiotorch.pipelines._wav2vec2.impl](https://pytorch.org/audio/stable/_modules/torchaudio/pipelines/_wav2vec2/impl.html#Wav2Vec2ASRBundle)<br>
[CTC Beam Search Decoder -- The Definition of the Decoder Class](https://pytorch.org/audio/stable/models.decoder.html#ctcdecoder)

In [1]:
import numpy
import torch
import torchaudio

from pyctcdecode import build_ctcdecoder
from torchaudio.models.decoder import ctc_decoder
from torchaudio.models.wav2vec2.utils import import_huggingface_model
from transformers import Wav2Vec2ProcessorWithLM, Wav2Vec2ForCTC, Wav2Vec2CTCTokenizer, Wav2Vec2FeatureExtractor

/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/torchaudio/_internal/module_utils.py:99: UserWarning: Failed to import soundfile. 'soundfile' backend is not available.
  warnings.warn("Failed to import soundfile. 'soundfile' backend is not available.")


In [2]:
print(torch.__version__)
print(torchaudio.__version__)

1.12.1+cu102
0.12.1+cu102


### **Data**

For this experiment, we'll use 4 audio files from the Common Voice 8.0 that were previously transcribed with mistakes (by several model setups) and several completely new audio files selected randomly from the Common Voice 11.0 that weren't previously seen by the model (4 of which can be categorized as pertaining to the medical sphere). (The audio files should be placed in the same directory as this notebook.)

In [3]:
wav_files = [# old files from Common Voice 8.0
             "common_voice_uk_21581951.wav",
             "common_voice_uk_21619415.wav",
             "common_voice_uk_27603609.wav",
             "common_voice_uk_27934776.wav",
             # new files from Common Voice 11.0
             "common_voice_uk_21565096.wav",
             "common_voice_uk_21565097.wav",
             "common_voice_uk_21585010.wav",
             "common_voice_uk_25154983.wav",
             "common_voice_uk_25154984.wav",
             "common_voice_uk_25155499.wav",
             "common_voice_uk_25652074.wav",
             "common_voice_uk_25652099.wav",
             "common_voice_uk_27278380.wav",
             "common_voice_uk_27278438.wav",
]

### **Inference with `transformers`**
Instantiate the model and its processor. The processor is composed of 3 integral components:
-  feature_extractor (processes speech signal to the model's input format)
-  tokenizer (processes the model's output format to text)
-  CTC decoder (with the Language Model (LM) support for the improved speech recognition decoding)
You can check the processor's components a bit more thoroughly and access, for instance, the tokenizer's vocabulary -- a set of possible predictions, which, in case of the ASR model, is an alphabet and some extra symbols. Next, you can see that the processor is instantiated with the default, fallback CTC decoder and the language model (LM) created from the dataset used for the model's training.<br>

(All 3 components can be instantiated separately and then passed to the processor as its parameters but, as you've already confirmed for yourself, there's no need in that because these components and their paramaters are automatically added and fully setup within the processor.)<br>
```python
from transformers import Wav2Vec2FeatureExtractor, Wav2Vec2CTCTokenizer

feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("Yehor/wav2vec2-xls-r-300m-uk-with-small-lm")
tokenizer = Wav2Vec2CTCTokenizer.from_pretrained("Yehor/wav2vec2-xls-r-300m-uk-with-small-lm")
decoder = build_ctcdecoder(...)

processor = Wav2Vec2ProcessorWithLM.from_pretrained(feature_extractor=feature_extractor, tokenizer=tokenizer, decoder=decoder)
```

In [4]:
model1 = Wav2Vec2ForCTC.from_pretrained("Yehor/wav2vec2-xls-r-300m-uk-with-small-lm")
processor = Wav2Vec2ProcessorWithLM.from_pretrained("Yehor/wav2vec2-xls-r-300m-uk-with-small-lm")

print(processor.feature_extractor)
print(processor.tokenizer)
print(processor.decoder)
print(processor.language_model)

# Extract the labels (the model's alphabet) from the tokenizer
labels = list(processor.tokenizer.get_vocab().keys())
print(labels)

/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/huggingface_hub/utils/_deprecation.py:97: FutureWarning: Deprecated argument(s) used in 'snapshot_download': allow_regex. Will not be supported from version '0.12'.

Please use `allow_patterns` and `ignore_patterns` instead.
  warnings.warn(message, FutureWarning)


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

Loading the LM will be faster if you build a binary file.
Reading /home/studio-lab-user/.cache/pyctcdecode/models--Yehor--wav2vec2-xls-r-300m-uk-with-small-lm/snapshots/bbd936400e7566ba44560440aa4abd05b5983c17/language_model/5gram_correct.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************


Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_side": "right",
  "padding_value": 0.0,
  "processor_class": "Wav2Vec2ProcessorWithLM",
  "return_attention_mask": true,
  "sampling_rate": 16000
}

PreTrainedTokenizer(name_or_path='Yehor/wav2vec2-xls-r-300m-uk-with-small-lm', vocab_size=38, model_max_len=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '[UNK]', 'pad_token': '[PAD]', 'additional_special_tokens': [AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True)]})
["'", '-', '[PAD]', '[UNK]', '|', 'а', 'б'

### **Experiment**

First, let's define a function for the greedy search algorithm, also known as **best-path decoding**.<br>
Next, let's set `beam_width=200`, and run the inference again. The default value for this parameter in the `pyctcdecode` decoder is 100. (**OBS!** The experiment has also been conducted for the `beam_width` of 1500, but there were no significant improvements in the transcription quality gained due to the beam saturation. In general, usually running your inference with the beams wider than 100-200 isn't recommended because that would cause a considerable decoding speed decrease.) You can also check the ASR model's predicted score and the LM's predicted score for each transcription as follows:
```python
print(processor.decode(outputs, beam_width=200).logit_score)
print(processor.decode(outputs, beam_width=200).lm_score)
```
Finally, let's check if attaching a custom LM affects the generated sentences in any major way. To initialize the CTC beam search decoder with a custom LM, you'll need:<br>
-  to generate a vocabulary file from the `unigrams.txt` file, which will trigger the lexicon-constrained beam search decoding (i.e. we'll instruct the model to reduce the search space down to the vocabulary, so that only words existing in it can be predicted)<br>
-  to create / or take a ready KenLM file (either `.arpa` or `.bin` one; the latter is recommended for faster loading).

The extra parameters, such as the alpha and beta weights, are usually tuned on a validation dataset. The alpha parameter is a weight for the LM; the beta is used to adjust the length score. **Additionally, you can view all the generated transcriptions for each audio file as follows**:
```python
print(lm_decoder.decode_beams(outputs))
```

In [5]:
def greedy_decode(logits):
    """Decode argmax of logits and squash in CTC fashion."""
    # Revert the dictionary that maps from letters and symbols
    # (labels) to indices, to map from indices to labels
    label_dict = {n: c for n, c in enumerate(labels)}
    prev_c = None
    out = []
    for n in logits.argmax(axis=1):
        # If not in labels, then assume it's a CTC blank character
        c = label_dict.get(n, "")
        if c != prev_c and c != "[PAD]":
            out.append(c)
        prev_c = c
    return "".join(out).replace("|", " ")

In [6]:
# GREEDY SEARCH DECODING
print("Test the greedy search decoding.")
transcriptions_greedy = []
for wav_file in wav_files:
    waveform, sample_rate = torchaudio.load(wav_file)
    resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000, resampling_method='sinc_interpolation')
    speech_array = resampler(waveform).squeeze().numpy()
    inputs = processor(speech_array, sampling_rate=16000, return_tensors="pt")['input_values']
    # Squeeze the batch_size dimension: [1, 149, 40] -> [149, 40],
    # to get the distribution of the labels for the entire sequence
    outputs = model1(inputs)['logits'].detach().numpy().squeeze(0)
    sequence = greedy_decode(outputs)
    transcriptions_greedy.append(sequence)
    print(sequence)

Test the greedy search decoding.
про це повідомляє през служба поліції київської області
зібрані дані застосовувалися поперше для гернерування адресної розсилки
хто розлягається огокворогого котрого втемрі віно не помітили 
буває професійні помічники домовляються з домовласником напавно ціну за квартиру 
перестаралося з корекцію вилиць 
про це свічить її зваложена й доглянута шкіра 
сім'ї зінвалідами іншим пільговиками постійно потребуватимуть комплексної державної підтримки
антибіотикорезистентний
антианемічний
до переваг цього методу належить простотатехнологічних схем і можливість творення замкнутих циклів
еч сіймач фегра і захистив титул
зверніть увагу жодної державністі московської не пахне
між частинами озброєним рушницями виблискували на сонці косарі і відділи озброєні косами 
рельєв хрепта сильно розчавлений вусчьовій зоні вирівнюється до переферій за рахунок поховання осить 


In [7]:
# BEAM SEARCH DECODING
print("\nTest the beam search decoding. (Default LM)")
transcriptions_beam_defaultlm = []
for wav_file in wav_files:
    waveform, sample_rate = torchaudio.load(wav_file)
    resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000, resampling_method='sinc_interpolation')
    speech_array = resampler(waveform).squeeze().numpy()
    inputs = processor(speech_array, sampling_rate=16000, return_tensors="pt")['input_values']
    # Squeeze the batch_size dimension: [1, 149, 40] -> [149, 40],
    # to get the distribution of the labels for the entire sequence
    outputs = model1(inputs)['logits'].detach().numpy().squeeze(0)
    sequence = processor.decode(outputs, beam_width=200).text
    transcriptions_beam_defaultlm.append(sequence)
    print(sequence)


Test the beam search decoding. (Default LM)
про це повідомляє пре служба поліції київської області
зібрані дані застосовувалися по перше для гернерування адресної розсилки
хто розлягається гокворогого котрого в темряві у не помітили
буває професійні помічники домовляються з домовласником напавно ціну за квартиру
перестаралася з корекцією вилиць
про це свідчить її зволожена й доглянута шкіра
сім'ї з інвалідами іншими пільговиками постійно потребуватимуть комплексної державної підтримки
антибіотикорезистентний
антианемічний
до переваг цього методу належить простота технологічних схем і можливість створення замкнутих циклів
едж свій мачфегра і захистив титул
зверніть увагу жодної державністі московською не пахне
між частинами озброєними рушницями виблискували на сонці косарі відділи озброєні косами
рельєф хребта сильно розчавлений в осьовій зоні вирівнюється до переферій за рахунок поховання осить


In [8]:
# Generate a vocab file from `unigrams.txt`
with open("unigrams.txt", mode='r', encoding='utf-8') as f:
    vocab_list = [t.lower() for t in f.read().strip().split('\n')]
    
lm_decoder = build_ctcdecoder(
    labels = labels,
    kenlm_model_path = '5gram.arpa',
    unigrams = vocab_list,
    # LM weight
    alpha = 0.5,
    # LM usage reward (never set to 0)
    beta = 2.0,
    # unk_score_offset=-10.0,
    # lm_score_boundary=True,
)

# Sanity check: we're using two different CTC decoders and, hence,
# two different language models
print(lm_decoder)
print(processor.decoder)

Loading the LM will be faster if you build a binary file.
Reading /home/studio-lab-user/sagemaker-studiolab-notebooks/asr_experiments/5gram.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
Found entries of length > 1 in alphabet. This is unusual unless style is BPE, but the alphabet was not recognized as BPE type. Is this correct?
****************************************************************************************************


In [9]:
# BEAM SEARCH WITH CUSTOM LANGUAGE MODEL
print("\nTest the beam search decoding. (Custom LM)")
transcriptions_beam_customlm = []
for wav_file in wav_files:
    waveform, sample_rate = torchaudio.load(wav_file)
    resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000, resampling_method='sinc_interpolation')
    speech_array = resampler(waveform).squeeze().numpy()
    inputs = processor(speech_array, sampling_rate=16000, return_tensors="pt")['input_values']
    # Squeeze the batch_size dimension: [1, 149, 40] -> [149, 40],
    # to get the distribution of the labels for the entire sequence
    outputs = model1(inputs)['logits'].detach().numpy().squeeze(0)
    sequence = lm_decoder.decode(outputs)
    transcriptions_beam_customlm.append(sequence)
    print(sequence)


Test the beam search decoding. (Custom LM)
про це повідомляє пре служба поліції київської області
зібрані дані застосовувалися по перше для гернерування адресної розсилки
хто розлягається гокворогого котрого в темряві у не помітили
буває професійні помічники домовляються з домовласником напавно ціну за квартиру
перестаралася з корекцією вилиць
про це свідчить її зволожена й доглянута шкіра
сім'ї з інвалідами іншими пільговиками постійно потребуватимуть комплексної державної підтримки
антибіотикорезистентний
антианемічний
до переваг цього методу належить простота технологічних схем і можливість створення замкнутих циклів
едж свій меч фегра і захистив титул
зверніть увагу жодної державністі московською не пахне
між частинами озброєними рушницями виблискували на сонці косарі відділи озброєні косами
рельєф хребта сильно розчавлений в осьовій зоні вирівнюється до переферій за рахунок поховання осить


### **Eror Analysis**
**OBS!** For each audio file, Line 1 -- greedy search, Line 2 -- beam search (default LM), Line 3 -- beam search (custom LM). Mistakes are marked in **bold**.

про це повідомляє пре**з** служба поліції київської області<br>
про це повідомляє **пре** служба поліції київської області<br>
про це повідомляє **пре** служба поліції київської області<br>
The beam search decoding didn't fix the greedy-search mistake, but only introduced a new one to replace it.<br>

зібрані дані застосовувалися **поперше** для ге**р**нерування адресної розсилки<br>
зібрані дані застосовувалися **по перше** для ге**р**нерування адресної розсилки<br>
зібрані дані застосовувалися **по перше** для ге**р**нерування адресної розсилки<br>
Here, the beam search didn't correct the spelling of the first word entirely, but it's closer to the proper spelling because it should be hyphened. This can be fixed by creating a new LM with the hyphened words -- it seems that right now the LM lexicon file misses such information.<br>

хто розлягається **огокворогого** котрого **втемрі** **віно** не помітили<br>
хто розлягається **гокворогого** котрого в темряві **у** не помітили<br>
хто розлягається **гокворогого** котрого в темряві **у** не помітили<br>
While two mistakes weren't corrected, there is a small improvement for the second mistake with the beam search.<br>

буває професійні помічники домовляються з домовласником **напавно** ціну за квартиру<br>
буває професійні помічники домовляються з домовласником **напавно** ціну за квартиру<br>
буває професійні помічники домовляються з домовласником **напавно** ціну за квартиру<br>
No improvements from the beam search on this transcription.<br>

перестарал**о**ся з корекцію вилиць<br>
перестаралася з корекцією вилиць<br>
перестаралася з корекцією вилиць<br>
The mistake (wrong gender-specific ending of the verb) is corrected by the beam search.<br>

про це **свічить** її зв**а**ложена й доглянута шкіра<br>
про це свідчить її зволожена й доглянута шкіра<br>
про це свідчить її зволожена й доглянута шкіра<br>
Both mistakes are corrected by the beam search.<br>

сім'ї **зінвалідами** **іншим** пільговиками постійно потребуватимуть комплексної державної підтримки<br>
сім'ї з інвалідами іншими пільговиками постійно потребуватимуть комплексної державної підтримки<br>
сім'ї з інвалідами іншими пільговиками постійно потребуватимуть комплексної державної підтримки<br>
Both mistakes are corrected by the beam search. However, it seems that all three transcripts miss the conjunction 'й' (and) between the words 3 and 4.<br>

антибіотикорезистентний<br>
антибіотикорезистентний<br>
антибіотикорезистентний<br>
No mistakes were made by any of the decoding methods.<br>

антианемічний<br>
антианемічний<br>
антианемічний<br>
No mistakes were made by any of the decoding methods.<br>

до переваг цього методу належить **простотатехнологічних** схем і можливість **творення** замкнутих циклів<br>
до переваг цього методу належить простота технологічних схем і можливість створення замкнутих циклів<br>
до переваг цього методу належить простота технологічних схем і можливість створення замкнутих циклів<br>
Both mistakes are corrected by the beam search.<br>

**еч** **сіймач** **фегра** і захистив титул<br>
едж свій **мачфегра** і захистив титул<br>
едж свій **меч фегра** і захистив титул<br>
While the most important error wasn't fixed by the beam search (there's no such words as 'фегра' / 'мачфегра', and the audio should have been transcribed as 'матч виграв' (won the match)), it still managed to at least get the first two words right.<br>

зверніть увагу жодн**ої** державніст**і** московськ**ої** не пахне<br>
зверніть увагу жодн**ої** державніст**і** московською не пахне<br>
зверніть увагу жодн**ої** державніст**і** московською не пахне<br>
Here, the endings of three words are incorrect (wrong case), and the beam search made the right agreement only in the third word (the mistake persists in the first two though).<br>

між частинами озброєним рушницями виблискували на сонці косарі **і** відділи озброєні косами<br>
між частинами озброєними рушницями виблискували на сонці косарі відділи озброєні косами<br>
між частинами озброєними рушницями виблискували на сонці косарі відділи озброєні косами<br>
The beam search removed the conjunction 'i', which mistakenly appeared in the transcript.<br>

рельє**в** хре**п**та сильно розчавлений **вусчьовій** зоні вирівнюється до пер**е**ферій за рахунок поховання **осить**<br>
рельєф хребта сильно розчавлений в осьовій зоні вирівнюється до пер**е**ферій за рахунок поховання **осить**<br>
рельєф хребта сильно розчавлений в осьовій зоні вирівнюється до пер**е**ферій за рахунок поховання **осить**<br>
Here, the greedy search made multople mistakes -- and three of them were corrected by the beam search -- ranging from misspelled words (orthographic mistakes) to misheards.<br>

### **Conclusion**
Compared to the greedy search, the transcripts transcribed by the `CTCBeanSearchDecoder` are definitely of higher quality. Even if the corrections were minor for some sentences, e.g. separating the words that were previously glued together by the greedy search, they can be deemed considerable since in some cases these corrections were enough to restore the sentence meaning. In addition to that, the model clearly uses the extra linguistic knowledge provided by the LM -- most of the mistakes fixed by the beam search decoder pertained to mispelling the prefixes / roots of the words, or incorrect agreement in case, number, and gender between the words.

### **Inference with `torchaudio`**
Import the original `transformers` Wav2Vec2.0 pretrained weights and convert the model to `torchaudio`'s format. The model is built of three components:
-  **feature_extractor** that extracts the acoustic features from the raw audio waveforms,
-  **encoder** that converts the extracted features into a sequence of probability distribution (expressed in negative log-likelihood) over the expected labels, and
-  **aux**, which is an auxiliary module; if specified, the encoder's output goes here. (In our case, it's the final projection layer of shape [in_features=1024, out_features=num_tokens.]
The decoder is a standalone module in `torchaudio` that has to be instantiated separately from the three above. You can access and inspect the model's components as shown below.

In [10]:
model2 = import_huggingface_model(model1)
print(model2.__class__)
# print(model2.feature_extractor)
# print(model2.encoder)
print(model2.aux)

<class 'torchaudio.models.wav2vec2.model.Wav2Vec2Model'>
Linear(in_features=1024, out_features=40, bias=True)


### **Decoder**
To be instantiated, the decoder requires loading the following data -- tokens (valid labels, i.e. letters + special symbols), lexicon (a dictionary of words and their spellings used to condition the model to only generate the words from this list), and a language model -- an N-gram model that contains probability scores expressed as logits for every known N-gram. Although this component is purely optional, using it signicantly boosts the decoding process and increases the transcription quality.

#### Tokens
These are a set of possible symbols that can be predicted by the ASR model, including letters of the alphabet, any other characters required by the orthographic rules of the language, and the blank and silent symbols. It can either be passed in as a file, where each line consists of the tokens corresponding to the same index, or as a list of tokens, each mapping to a unique index.<br>
**OBS!** Since we're using a converted HuggingFace `Wav2Vec2` model, we can't use the default token values of the `torchaudio` `CTCDecoder` and have to modify these. According to the `torchaudio` documentation, the first two symbols in `tokens.txt` are usually the blank and silent character, and their default values are typically "**-**" (hyphen) and "**|**" (vertical bar) respectively. In `transformers`, in the original `Wab2Vec2` `CTCTokenizer` implementation, "|" is used as a word delimiter whereas "[PAD]" is a pad and a blank token that serves as a space between the letters within a single word. The blank token in `torchaudio` is configurable. Since Ukrainian spelling makes use of the "**-**" (hyphen) and "**'**" (apostrophe) symbols, we have to assign our `tokens.txt` and assign the "[PAD]" as a new value for the blank character. __You can't add, delete, or move the tokens around your file / list because all the symbols appear there in a fixed order. Therefore, changing it will break the original mapping from each token to its respective index and, as a consequence, ruin the decoding.__

#### Lexicon
The lexicon is a mapping from words to their corresponding tokens sequence, and is used to restrict the search space of the decoder to only words from the lexicon. The expected format of the lexicon file is a line per word, with a word followed by its space-split tokens. Conventionally, a word is separated from its sequence of tokens by a tab or a space.<br>
The lexicon can be automatically generated from the `unigrams.txt` but you have to thoroughly inspect it and remove any lines that doesn't constitute a valid word. Otherwise, initialzing the decoder will throw an error. 

#### Language Model
A language model can be used in decoding to improve the results, by factoring in a language model score that represents the likelihood of the sequence into the beam search computation.<br>
**No Language Model**: To create a decoder instance without a language model, set `lm=None` when initializing the decoder.<br>
**KenLM**: This is an n-gram language model trained with the KenLM library. Both the `.arpa` or the binarized `.bin` LM can be used, but the binary format is recommended for faster loading.

The most important parameters to consider when initializing the beam-search CTC decoder are listed below. These can significantly impact the decoder's performance, so choosing their values can be a delicate process. Since all improvements come at their own cost, configuring these parameters will largely depend on the sacrifices you're ready to make in order to achieve the set goals, i.e. generating higher-quality transcriptions will definitely use more computational resources, so it will also take longer time to decode the model's outputs.

#### `nbest`
This parameter indicates the number of best hypotheses to return as well as their final logit scores, which is often beneficial while configuring the most optimal values for the inference. For instance, by setting `nbest=5` when building the beam search decoder, you'll be able to access the hypotheses with the top 5 scores.

```python
for i in range(5):
    transcript = " ".join(beam_search_result[0][i].words).strip()
    score = beam_search_result[0][i].score
```

#### `beam size`
This parameter determines the maximum number of best hypotheses to hold after each decoding step. Using larger beam sizes allows for exploring a larger range of possible hypotheses which can produce hypotheses with higher scores, but it's computationally more expensive and does not provide additional gains beyond a certain point (often, using a beam size of 1500 provides the same output as a beam size of 200).

```python
beam_sizes = [50, 200, 500, 1500]

for beam_size in beam_sizes:
    beam_search_decoder = ctc_decoder(
        lexicon='lexicon.txt',
        tokens='tokens.txt',
        lm='5gram.arpa',
        beam_size=beam_size,
        lm_weight=LM_WEIGHT,
        word_score=WORD_SCORE,
    )

    print_decoded(beam_search_decoder, emission, "beam size", beam_size)
```

#### `beam size token`
This parameter corresponds to the number of tokens to consider for expanding each hypothesis at the decoding step. Exploring a larger number of next possible tokens increases the range of potential hypotheses, but yet again at the cost of computation.

```python
num_tokens = len(tokens)
beam_size_tokens = [1, 5, 10, num_tokens]

for beam_size_token in beam_size_tokens:
    beam_search_decoder = ctc_decoder(
        lexicon='lexicon.txt',
        tokens='tokens.txt',
        lm='5gram.arpa',
        beam_size_token=beam_size_token,
        lm_weight=LM_WEIGHT,
        word_score=WORD_SCORE,
    )

    print_decoded(beam_search_decoder, emission, "beam size token", beam_size_token)
```

#### `beam threshold`
This parameter is used to prune the stored hypotheses set at each decoding step, removing hypotheses whose scores are greater than `beam_threshold` away from the highest scoring hypothesis. There's a balance between choosing smaller thresholds to prune more hypotheses and reduce the search space, and choosing a large enough threshold such that plausible hypotheses are not pruned.

```python
beam_thresholds = [1, 5, 10, 25]

for beam_threshold in beam_thresholds:
    beam_search_decoder = ctc_decoder(
        lexicon='lexicon.txt',
        tokens='tokens.txt',
        lm='5gram.arpa',
        beam_threshold=beam_threshold,
        lm_weight=LM_WEIGHT,
        word_score=WORD_SCORE,
    )

    print_decoded(beam_search_decoder, emission, "beam threshold", beam_threshold)
```

#### `language model weight`
This parameter is the weight to assign to the LM score which to accumulate with the ASR model score for determining the overall scores. Larger weights encourage the model to predict next words based on the language model, while smaller weights give more weight to the acoustic model score instead.

```python
lm_weights = [0, 5, 10]

for lm_weight in lm_weights:
    beam_search_decoder = ctc_decoder(
        lexicon='lexicon.txt',
        tokens='tokens.txt',
        lm='5gram.arpa',
        lm_weight=lm_weight,
        word_score=WORD_SCORE,
    )

    print_decoded(beam_search_decoder, emission, "lm weight", lm_weight)
```

In [11]:
# Create the `tokens.txt` file
# with open('tokens.txt', mode='w', encoding='utf-8') as f:
#     for label in labels:
#         f.write("{}\n".format(label))

# Create the `lexicon.txt` file
# with open('unigrams.txt', mode='r', encoding='utf-8') as f1:
#     with open('lexicon.txt', mode='w', encoding='utf-8') as f2:
#         for word in f1:
#             word = word.strip()
#             spacesplit_word = ' '.join(word)
#             f2.write("{0}\t{1} |\n".format(word, spacesplit_word))

### **Greedy Decoder**

In [12]:
class GreedyCTCDecoder(torch.nn.Module):
    def __init__(self, labels, blank=0):
        super().__init__()
        self.labels = labels
        self.blank = blank

    def forward(self, emission):
        indices = torch.argmax(emission, dim=-1)
        indices = torch.unique_consecutive(indices, dim=-1).squeeze().tolist()
        sequence = "".join([self.labels[i] for i in indices]).replace("[PAD]", "")
        # .replace("|", " ")
        return sequence

greedy_decoder = GreedyCTCDecoder(labels)
print(greedy_decoder)
print(labels)
print(greedy_decoder.blank)

GreedyCTCDecoder()
["'", '-', '[PAD]', '[UNK]', '|', 'а', 'б', 'в', 'г', 'д', 'е', 'ж', 'з', 'и', 'й', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф', 'х', 'ц', 'ч', 'ш', 'щ', 'ь', 'ю', 'я', 'є', 'і', 'ї', 'ґ', '<s>', '</s>']
0


In [13]:
# GREEDY SEARCH DECODING
print("Test the greedy search decoding.")
transcriptons_greedy = []
for wav_file in wav_files:
    waveform, sample_rate = torchaudio.load(wav_file)
    resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000, resampling_method='sinc_interpolation')
    features = resampler(waveform)
    with torch.inference_mode():
        emissions, _ = model2(features)
        # Normalizing the model's output
        emissions = torch.log_softmax(emissions, dim=-1).cpu().detach()
        predicted_ids = greedy_decoder(emissions[0])
        sequence= "".join(predicted_ids).replace("|", " ")
        transcriptions_greedy.append(sequence)
        print(sequence)

Test the greedy search decoding.
про це повідомляє през служба поліції київської області 
зібрані дані застосовувалися поперше для гернерування адресної розсилки
хто розмягається ог ворогого ко трого темля віно не томітили 
буває професійні помічники домовляються з дом о власником напавноційну за квартиру 
перестаралася з корекцію вилиць 
процесвічить її зваложена й доглянута шкіра
сім'ї зінвалідими іншим пільговиками постійно потребуватимуть комплексної державної підтримки
антибіотикорезистентний
антианемічний
до переваг цього методу належить простота технологічних схем і можливість творення замкнутих циклів
еж сіймач вигра і захистив титул
зверніть увагу жодної державністі московської не пахне
між частинами озброєними рушницями ви блискували на сонці косарі відділи озброєні косари 
рельєф хрепта сильно розчавлений всовій зоні вирівнюється до переперіг за рахунок поховання осодь 


In [14]:
beam_search_decoder = ctc_decoder(
    lexicon='lexicon.txt',
    tokens='tokens.txt',
    lm="5gram.arpa",
    nbest=3,
    beam_size=200,
    lm_weight=3.5,
    word_score=-0.3,
    blank_token = "[PAD]",
    sil_token = "|",
    unk_word = "[UNK]",
)
print(beam_search_decoder)
# Check whether the decoder's blank character was set properly:
# index = 2 is the "[PAD]" tokem, as required.
print(beam_search_decoder.blank)

Loading the LM will be faster if you build a binary file.
Reading 5gram.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************


2


In [15]:
# BEAM SEARCH DECODING
print("Test the beam search decoding.")
transcriptions_beam = []
for wav_file in wav_files:
    waveform, sample_rate = torchaudio.load(wav_file)
    resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000, resampling_method='sinc_interpolation')
    speech = resampler(waveform)
    with torch.inference_mode():
        emissions, _ = model2(speech)
        # Normalizing the model's output
        emissions = torch.log_softmax(emissions, dim=-1)
        sequence = beam_search_decoder(emissions.cpu().detach())
        sequence = " ".join(sequence[0][0].words).strip()
        transcriptions_beam.append(sequence)
    print(sequence)

Test the beam search decoding.
про це повідомляє прес-служба поліції київської області
зібране дані застосовуватися по-перше для переривання адресою носилки
хто розлягається ворожого котрого в темряві мене помітили
буває професійні помічники домовляються з домовласником на певну ціну за квартиру
перестаралася з корекцією вилиць
про це свідчить її зволожена й доглянута шкіра
сім'ї з інвалідами іншими пільговиками постійно потребуватимуть комплексної державної підтримки
антибіотикорезистентний
антианемічний
до переваг цього методу належить простота технологічних схем і можливість створення замкнутих циклів
піймав виграв і захистив титул
зверніть увагу жодною державністю московською не пахне
між частинами озброєними рушницями виблискували на сонці косарі відділи озброєні косами
рельєф хребта сильно розчавленій свій зоні вирівнюється до периферії за рахунок поховання осадів


### **Error Analysis**

OBS! For each audio file, Line 1 -- greedy search, Line 2 -- beam search (`beam_size=200`), Line 3 -- beam search (`beam_size=500`), Line 4 -- beam search (beam_size=1500). Mistakes are marked in bold.

про це повідомляє пре**з**служба поліції київської області<br>
про це повідомляє прес-служба поліції київської області<br>
про це повідомляє прес-служба поліції київської області<br>
про це повідомляє прес-служба поліції київської області<br>
With as few as 200 beams, the beam search decoding allowed to fix the mistakes -- find a correct root consonant for the word 'прес' (press) and add a hyphen between the words 'прес' and 'служба' (service). (The default value of the number of beams to utilize is 50.)<br>

зібрані дані застосовувалися **поперше** для ге**р**нерування адресної розсилки<br>
зібран**е** дані застосовуватися по-перше для **мене вання** адрес**ою** **но**силки<br>
зібран**е** дані застосовуватися по-перше для **переривання** **адресою** **носилки**<br>
зібран**е** дані застосовуватися по-перше для **переривання** **адресою** **носилки**<br>
This sample is a sad example of how the beam search can go rogue -- it made mistakes in every word, except words #4 and #5. The errors in the last 3 words are especially grave because the generated words aren't even close to the reference transcript. However, the LM seems to be working perfectly fine -- we no longer have the issues with the words that should be hyphened, e.g. 'по-перше' (firstly).<br>

хто розлягаєтьця **ог** в**о**ртового **ктрого** **втемля** **віноне** помітили<br>
хто розлягається **ворожого** котрого в темряві **ме**не помітили<br>
хто розлягається **готового** котрого в темряві **не** не помітили<br>
хто розлягається **готового** котрого в темряві **ме**не помітили<br>
Unfortunately, the beam search decoder didn't achieve much in this sentence either (still issues with the word #3: 'готового' should be 'гук вартового', and the word #7: 'мене' should be 'ми не'.<br>
 
буває професійні помічники домовляються **здом** **о**власником на п**а**вну ціну за квартиру<br> 
буває професійні помічники домовляються з домовласником на певну ціну за квартиру<br>
буває професійні помічники домовляються з домовласником на певну ціну за квартиру<br>
буває професійні помічники домовляються з домовласником на певну ціну за квартиру<br>
Here, the beam search decoder fixed the transcription mistakes made by the greedy search decoder using just 200 paths, again.<br>

перестаралася з корекці**ю** вилиць<br>
перестаралася з корекцією вилиць<br>
перестаралася з корекцією вилиць<br>
перестаралася з корекцією вилиць<br>
The greedy search transcribed an incorrect case ending for the word #3 'корекцію' (correction; a term used in plastic surgery), and the beam search corrected it.<br>

**проц**е **свічить** її зв**а**ложена й доглянута шкіра<br>
про це свідчить її зволожена й доглянута шкіра<br>
про це свідчить її зволожена й доглянута шкіра<br>
про це свідчить її зволожена й доглянута шкіра<br>
Again, all the mistakes made by the greedy search were corrected by the beam search.<br>

сім'ї **зінвалідами** іншим пільговиками постійно потребуватимуть комплексної державної підтримки<br>
сім'ї з інвалідами іншими пільговиками постійно потребуватимуть комплексної державної підтримки<br>
сім'ї з інвалідами іншими пільговиками постійно потребуватимуть комплексної державної підтримки<br>
сім'ї з інвалідами іншими пільговиками постійно потребуватимуть комплексної державної підтримки<br>
Everything is transcribed perfectly in this example.<br>

антибіотикорезистентний<br>
антибіотикорезистентний<br>
антибіотикорезистентний<br>
антибіотикорезистентний<br>
Correct. (The word means 'antibiotic-resistant'.)<br>

антианемічний<br>
антианемічний<br>
антианемічний<br>
антианемічний<br>
Correct. (The word means 'antianemic'.)<br>

до переваг цього методу належить простота технологічних схем і можливість **творення** замкнутих циклів<br>
до переваг цього методу належить простота технологічних схем і можливість створення замкнутих циклів<br>
до переваг цього методу належить простота технологічних схем і можливість створення замкнутих циклів<br>
до переваг цього методу належить простота технологічних схем і можливість створення замкнутих циклів<br>
The beam search correctly added a prefix to the word #11 ('творення' -> 'створення').<br>

**ечсвіймач вегра** і захистив титул<br>
**сімей** виграв і захистив титул<br>
едж свій матч виграв і захистив титул<br>
едж свій матч виграв і захистив титул<br>
This sentence was a big problem for the `transformers` beam search -- no amount of beams was enough to correctly transcribe the first 4 words ('едж свій матч виграв' reads 'Edge [surname] won his match'). The beam search provided via `torchaudio` finally achieves this goal with 500 beams.<br>

зверніть увагу жодн**ої** державніст**і** московськ**ої** не пахне<br>
зверніть увагу жодною державністю московською не пахне<br>
зверніть увагу жодною державністю московською не пахне<br>
зверніть увагу жодною державністю московською не пахне<br>
The beam search algorithm corrects the case endings for the words #3, #4, and #5.<br>

між частинами озброєни**м** рушницями **ви блискували** на сонці косарі відділи озброєні косари<br>
між частинами озброєними рушницями виблискували на сонці косарі відділи озброєні косами<br>
між частинами озброєними рушницями виблискували на сонці косарі відділи озброєні косами<br>
між частинами озброєними рушницями виблискували на сонці косарі відділи озброєні косами<br>
There are no orthographic mistakes in the transcriptions generated by the beam search but this sentence is a prime example of how the punctuation is essentially ignored by many ASR models: the last 3 words constitute a clarifying clause that describes the 4th word from the end in more details and, thus, should be separated from it by either a comma or a dash.<br>

рельє**в** хре**п**та сильно розчавлений **в сьовій** зоні вирінюється до **переперіг** за р**о**хунок поховання **осоді**<br>
рельєф хребта сильно розчавленій свій зоні вирівнюється до периферії за рахунок поховання осадів<br>
рельєф хребта сильно розчавленій **о**сьовій зоні вирівнюється до периферії за рахунок поховання осадів<br>
рельєф хребта сильно розчавленій **о**сьовій зоні вирівнюється до периферії за рахунок поховання осадів<br>
In the final example, the beam search algorithm brilliantly fixes all the spelling mistakes. One of them persistently remained a problem for the `transformers` beam search -- it kept spelling 'переферії' instead of the correct 'периферії', even after attaching 2 different LMs to it. However, the `torchaudio` beam search powered by the LM solved the issue. The only mistake made by the beam search here is the omitted preposition between the words #4 and #5 (it's difficult to predict whether the NMT model is sensitive to that or not.)<br>

### **Conclusion**
Although after using the `torchaudio` beam search decoder, there still are transcribing errors left in some sentences, my overall judgement is that it presented itself as a robust and reliable tool. Using it is certainly beneficial for the ASR task, and the ASR model performs especially well when boosted by the external linguistic data (mainly, the spelling dictionary and the language model). At certain points, the `torchaudio` beam-search CTC decoder outperformed its competitor, the `transformers` beam-search CTC decoder, which indicates that we should consider it as the primary CTC decoder for any our future ASR work, in case we choose to utilize the beam search algorithm in it.